In [ ]:
#>> Projeto 1 de Grafos 
#>> Disciplina: Teoria dos Grafos — UnB
#>> Turma: 01, 2025/2
#>> Professor: Dibio
#>>> Integrantes:
#>- Julia Paulo Amorim - 241039270
#>- Leticia Gonçalves Bomfim - 241002411
#>- Vitor Alencar Ribeiro - 231036292

#>>>>>> Aqui descreveremos as instruções de quais extensões devem ser baixadas e como rodar o projeto para Linux
    #1 - Verificar se python está instalado com python3 --version
    #2  - Instalar o pip com sudo apt install python3-pip
    #3 - Crie um ambiente virtual (se quiser) para rodar o projeto com python3 -m venv venv
    #4 - Ative o ambiente virtual com source venv/bin/activate
    #5 - Instale as dependências do projeto com pip install networkx numpy matplotlib community requests
    #6 - Para rodar o projeto, utilize python3 facebook.py
    #7 - Já em relação ao relatório, baixe o Jupyter notebook com pip install jupyterlab
    #8 - Para iniciar o Jupyter, inicie o comando jupyter lab

#Imports do grafo:
import networkx as nx  #Manipula gráficos
import requests  #Faz o download de dados por HTTP
import numpy as np  #Auxilia nas operações numéricas
import matplotlib.pyplot as plt  #Possibilita a visualização dos gráficos
import community as community_louvain  #Detecta as comunidades
from matplotlib.patches import Patch  #Possibilita a criação das legendas personalizadas

#Etapa 1 - Coleta de Dados:

class FacebookGraph:
    #> O processo envolve o download do arquivo compactado de arestas e a gravação local em disco, garantindo que o grafo possa ser carregado nas etapas seguintes.
        #>> Como a Etapa Foi Implementada:
    #>> -Download Automático: O método baixar_dados() utiliza uma requisição HTTP para acessar diretamente o dataset hospedado no site oficial da Universidade de Stanford
    #>> (SNAP), onde está armazenado o grafo "facebook_combined.txt.gz".
    #>> -Verificação de Sucesso: Caso o download seja bem-sucedido, o arquivo é salvo localmente e uma mensagem de confirmação é exibida; caso contrário, o erro é
    #>> tratado e descrito no terminal.
        #>> Bibliotecas Escolhidas:
    #>> -Requests: A biblioteca `requests` é a ferramenta padrão e mais estável em Python para comunicações HTTP. Foi escolhida por sua simplicidade, confiabilidade e por
    #>> permitir o tratamento direto de exceções (com o método raise_for_status), evitando falhas silenciosas durante o download.
    
    def __init__(self):
        self.G = None
        self.G_subset = None

    def baixar_dados(self):
        url = "https://snap.stanford.edu/data/facebook_combined.txt.gz"

        try:
            #Requests é uma biblioteca cliente HTTP para Python
            response = requests.get(url)
            response.raise_for_status()

            #Abre o arquivo com as arestas
            with open("facebook_combined.txt.gz", "wb") as f:
                f.write(response.content)
            
            print("Arquivo aberto com sucesso!\n")
            
            return True
        
        except Exception as e:
            print(f"Não foi possível abrir o arquivo: {e}")
            return False

#Etapa 2 - Construir o grafo:
     #>> Esta etapa cumpre a segunda parte do projeto: transformar os dados baixados em uma estrutura de grafo manipulável e gerar um subgrafo com 2000 nós para análises
     #>> subsequentes.
        #>> Como a Etapa Foi Implementada:
    #>> -Carregamento da Rede: O método carregar_rede() usa uma função nativa do NetworkX que interpreta automaticamente o arquivo de lista de arestas (.txt.gz) e cria um
    #>> grafo não direcionado. Em seguida, são impressas métricas básicas — número de nós, arestas e densidade — que servem como uma validação inicial do dataset.
    #>> -Seleção Aleatória de Nós: O método extrair_subconjunto() faz uma amostragem aleatória de 2000 vértices do grafo principal. Essa abordagem reduz a complexidade
    #>> computacional e permite que o restante da análise (centralidades e comunidades) seja executado de forma eficiente.
    #>> -Construção do Subgrafo: Após a seleção, é criado um subgrafo contendo apenas os nós sorteados e suas arestas internas. Assim, garante-se que o subconjunto
    #>> preserve a estrutura original de conexões entre os nós escolhidos.
        #>> Bibliotecas Escolhidas:
    #>> -NetworkX: É a principal biblioteca Python para modelagem e análise de grafos. Ela permite ler o arquivo de arestas diretamente e fornece métodos prontos
    #>> para criar subgrafos, calcular densidade e realizar inspeções estruturais.
    #>> -NumPy: Utilizada aqui pela função `np.random.choice`, que oferece uma maneira eficiente e reproduzível de selecionar nós aleatórios sem repetição
    #>> (replace=False), garantindo uma amostragem equilibrada da rede.
        
    def carregar_rede(self):
        try:
            #Função nativa do NetworkX que lê um arquivo de lista de arestas e cria um grafo
            self.G = nx.read_edgelist("facebook_combined.txt.gz")

            #Teste se criou o grafo corretamente
            print(f" - Nós: {self.G.number_of_nodes()}")          #4039
            print(f" - Arestas: {self.G.number_of_edges()}")      #88234
            print(f" - Densidade: {nx.density(self.G):.6f}")      #0.010820

            return True
        
        except Exception as e:
            print(f"Erro ao criar grafo: {e}")
            return False
            
        #Parte da Etapa 1 - Extrair desse grafo carregado 2 mil vértices
        
    def extrair_subconjunto(self, n_nos=2000):
        if self.G is None:
            print("O grafo não existe!")
            return False
        
        todos_nos = list(self.G.nodes())
        #Utiliza a biblioteca numpy para escolher nós aleatórios dentre os da lista
        #replace=False é para não haver repetição de nós
        nos_selecionados = np.random.choice(todos_nos, size=n_nos, replace=False)

        #G_subset é o subgrafo com os 2000 selecionados
        self.G_subset = self.G.subgraph(nos_selecionados).copy()

        for node in nos_selecionados:
            vizinhos = list(self.G.neighbors(node))
            for vizinho in vizinhos:
                if vizinho in nos_selecionados:
                    self.G_subset.add_edge(node, vizinho)

        print("\nInformações gerais dos 2000 nós selecionados:")
        print(f" - Nós: {self.G_subset.number_of_nodes()}")
        print(f" - Arestas: {self.G_subset.number_of_edges()}")
        print(f" - Densidade: {nx.density(self.G_subset):.6f}")

        return True

#Etapa 3 - Extrair do grafo as métricas:
    #>> Este bloco de código foi desenvolvido para cumprir a Etapa 3 do projeto, que exige a extração de métricas de centralidade e o mapeamento de
    #>> comunidades da rede social. A implementação foi feita dentro do método calcular_metricas() para manter a organização e a modularidade do código.7
    
     #>> Como a Análise Foi Implementada: O método calcular_metricas() segue uma lógica sequencial e clara:
        #>> -Centralização: Todas as análises foram agrupadas em uma única função para facilitar a execução e a manutenção do código.
        #>> -Cálculo das Métricas: Para cada uma das quatro medidas de centralidade (Grau, Intermediação, Proximidade e Autovetor), o código chama uma função
        #>> específica e otimizada da biblioteca NetworkX.
        #>> -Identificação dos Nós Influentes: Após cada cálculo, o código ordena os resultados e imprime no terminal os 5 nós mais importantes (o "Top 5")
        #>> para aquela métrica. Isso fornece uma visão imediata de quais "usuários" são mais influentes em diferentes aspectos da rede, informação crucial
        #>> para o relatório final.
        #>> -Detecção de Comunidades: Utiliza-se a biblioteca community para aplicar o algoritmo de Louvain, conforme exigido pelo enunciado do projeto,
        #>> identificando os principais agrupamentos ("panelinhas") na rede.
        #>> -Armazenamento: Todos os resultados são salvos em variáveis da classe, permitindo que sejam facilmente acessados por outras funções, como a de
        #>> visualização, para criar os grafos coloridos e rotulados.

     #>> Bibliotecas Escolhidas:
        #>>A escolha das bibliotecas foi baseada nas que mais fazem sentido nos requisitos do projeto:
        #>> -NetworkX: É a biblioteca padrão-ouro para análise de grafos em Python, [cite_start]explicitamente recomendada nas instruções do trabalho[cite: 1158]. A principal
        #>> vantagem é que ela já possui implementações robustas e academicamente validadas de todos os algoritmos de centralidade necessários. Isso elimina a necessidade de programar 
        #>> essas complexas rotinas matemáticas do zero, garantindo a precisão dos resultados e a simplicidade do código.
        #>> -Python-louvain (importada como community): O enunciado do projeto exige [cite_start]especificamente o "Mapeamento de comunidades pelo algoritmo de Louvain"[cite: 1150].
        #>> A biblioteca python-louvain é a implementação padrão deste método para Python e se integra perfeitamente com objetos de grafo do NetworkX.
        #>> A escolha, portanto, foi a mais direta para cumprir este requisito.


    def calcular_metricas(self, top_n=5):
        #Calcula as métricas de centralidade e detecta comunidades no subgrafo.
        if self.G_subset is None:
            print("O subgrafo não existe! Execute extrair_subconjunto() primeiro.")
            return False
        
        print("\n--- CALCULANDO MÉTRICAS DE CENTRALIDADE E COMUNIDADES ---")
        
        try:
            #Degree Centrality
            print("\n[1] Grau de Centralidade (Degree):")
            print("Mede a popularidade de um nó pelo número de conexões diretas.")
            degree_centrality = nx.degree_centrality(self.G_subset)
            sorted_degree = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
            print(f"  Top {top_n} nós com maior Grau de Centralidade:")
            for i in range(top_n):
                print(f"    {i+1}. Nó {sorted_degree[i][0]}: {sorted_degree[i][1]:.4f}")

            #Betweenness Centrality
            print("\n[2] Centralidade de Intermediação (Betweenness):")
            print("Mede a importância de um nó como 'ponte' nos caminhos mais curtos entre outros nós.")
            betweenness_centrality = nx.betweenness_centrality(self.G_subset)
            sorted_betweenness = sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)
            print(f"  Top {top_n} nós com maior Centralidade de Intermediação:")
            for i in range(top_n):
                print(f"    {i+1}. Nó {sorted_betweenness[i][0]}: {sorted_betweenness[i][1]:.4f}")

            #Closeness Centrality
            print("\n[3] Centralidade de Proximidade (Closeness):")
            print("Mede quão rápido um nó consegue alcançar todos os outros na rede.")
            closeness_centrality = nx.closeness_centrality(self.G_subset)
            sorted_closeness = sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)
            print(f"  Top {top_n} nós com maior Centralidade de Proximidade:")
            for i in range(top_n):
                print(f"    {i+1}. Nó {sorted_closeness[i][0]}: {sorted_closeness[i][1]:.4f}")

            #Eigenvector Centrality
            print("\n[4] Centralidade de Autovetor (Eigenvector):")
            print("Mede a influência de um nó com base na importância de seus vizinhos.")
            try:
                eigenvector_centrality = nx.eigenvector_centrality(self.G_subset, max_iter=1000)
                sorted_eigenvector = sorted(eigenvector_centrality.items(), key=lambda item: item[1], reverse=True)
                print(f"  Top {top_n} nós com maior Centralidade de Autovetor:")
                for i in range(top_n):
                    print(f"    {i+1}. Nó {sorted_eigenvector[i][0]}: {sorted_eigenvector[i][1]:.4f}")
            except nx.PowerIterationFailedConvergence:
                print("  Cálculo de autovetor não convergiu. Pulando esta métrica.")

            #Algoritmo de Louvain
            print("\n[5] Mapeamento de Comunidades (Louvain):")
            print("Agrupa os nós em 'panelinhas' onde as conexões internas são mais fortes.")
            communities = community_louvain.best_partition(self.G_subset)
            num_communities = len(set(communities.values()))
            print(f"  Número de comunidades detectadas: {num_communities}")

            # Armazenando os resultados na classe para uso posterior
            self.centrality_measures = {
                'degree': degree_centrality,
                'betweenness': betweenness_centrality,
                'closeness': closeness_centrality,
                'eigenvector': eigenvector_centrality if 'eigenvector_centrality' in locals() else None
            }
            self.communities = communities
            
            return True
            
        except Exception as e:
            print(f"Erro ao calcular as métricas: {e}")
            return False

    #>> Como as Medidas São Calculadas?

       #>> -Grau de Centralidade (Degree): O NetworkX realiza a contagem mais direta: para cada nó, ele simplesmente conta quantas arestas (conexões) estão
    #>> ligadas a ele. Em uma rede social, isso mede a "popularidade" de um usuário pelo seu número de amigos diretos.
        #>> -Centralidade de Intermediação (Betweenness): O algoritmo identifica o caminho mais curto entre todos os pares de nós possíveis na rede. Em
    #>> seguida, ele verifica cada nó individualmente e conta em quantos desses caminhos mais curtos ele aparece. Um nó com alta intermediação funciona
    #>> como uma "ponte", sendo vital para a comunicação entre diferentes grupos da rede.
        #>> -Centralidade de Proximidade (Closeness): O algoritmo calcula a "distância média" de um nó para todos os outros nós da rede. Um nó com alta
    #>> proximidade é aquele que precisa de poucos "saltos" para alcançar qualquer outra pessoa. É um nó posicionado de forma ideal para espalhar
    #>> informações rapidamente por toda a rede.
        #>> -Centralidade de Autovetor (Eigenvector): Esta medida calcula a influência de um nó de forma recursiva. A ideia é que ter muitas conexões não é tudo,
    #>> o mais importante é estar conectado a pessoas que também são importantes.O algoritmo atribui uma pontuação a cada nó baseada na soma das
    #>> pontuações de seus vizinhos. Um nó com alto autovetor é um verdadeiro "influenciador", pois está conectado a outros influenciadores.
        #>> -Mapeamento de Comunidades (Louvain): O algoritmo de Louvain é um método iterativo que busca a melhor "divisão" da rede em comunidades. Ele faz
    #>> isso tentando maximizar uma métrica chamada "modularidade", que é alta quando há muitas conexões "dentro" de um grupo e poucas conexões "entre"
    #>> grupos diferentes. Ele agrupa os nós de forma a encontrar as "panelinhas" mais coesas e bem definidas da rede social.


    #Etapa 4 - Visualizar o grafo e as medidas extraídas
    def visualizar_rede(self):
        try:
            if self.G_subset is None:
                print("O subgrafo não existe!")
                return False

    #>> A biblioteca matplotlib é uma biblioteca do Python ideal para criar visualizações matemáticas, a matplotlib.pyplot é uma coleção de funções específicas para plotar grafos e 
    #>> gráficos.
    #>> A função plt.figure cria uma imagem.

            plt.figure(figsize=(20, 15))

    #>> A função spring_layout é nativa do NetworkX e distribui os nós a partir do algoritmo de força direcionada de Fruchterman-Reingold
    #>> A utilização dessa distribuição para os nós foi por questão estética, o algoritmo precisa do número k que é a distância entre os nós e o número de iterações para o algoritmo 
    #>> convergir. A escolha de k=2 e iterations=1000 foram para deixar uma distribuição mais organizada.

            pos = nx.spring_layout(self.G_subset, k=2, iterations=2000)

    #>> O tamanho dos nós é variável de acordo com o grau dele, mas para a diferença visual não ser tão grande, foi utilizado o logaritmo do grau dos nós para criar uma lista com os 
    #>> tamanhos. O log()+1 evita erros com o log(0) que é indefinido.

            #Calcula o grau de cada nó coloca em um dicionário e o tamanho do nó é proporcional ao grau
            graus = dict(self.G_subset.degree())
            tamanhos = [np.log(graus[node]+1)*25 for node in self.G_subset.nodes()]

    #>> A função percentile da biblioteca NumPy divide os graus em valores de porcentagem. Para os nós serem dividos em cores de acordo com o grau, o percentis divide os valores dos 
    #>> graus em porcentagens de 25%, 50%, 75% e 95%. O que significa que se 25% dos nós tem um certo valor de grau ou menor eles vão ter a mesma cor, nós utilizamos a paleta de 
    #>> cores "plasma" do matplotlib que faz com que os nós menos conectados sejam da cor azul e os mais da cor amarela.

            #Para melhor visualização no grafo, os nós tem cores de acordo com o grau
            graus_valores = list(graus.values())
            percentis = np.percentile(graus_valores, [25, 50, 75, 95])

    #>> Divide as cores de cada nó em 4 divisões de acordo com as porcentagens calculadas anteriormente

            node_colors = []
            for node in self.G_subset.nodes():
                if graus[node] <= percentis[0]:
                    node_colors.append(0)
                elif graus[node] <= percentis[1]:
                    node_colors.append(1)
                elif graus[node] <= percentis[2]:
                    node_colors.append(2)
                elif graus[node] <= percentis[3]:
                    node_colors.append(3)
                else:
                    node_colors.append(4)
            
            #Definições de visualização
            nx.draw_networkx_nodes(self.G_subset, pos, node_size=tamanhos, node_color=node_colors, cmap="plasma", alpha=1)
            nx.draw_networkx_edges(self.G_subset, pos, alpha=1, edge_color="gray", width=0.5)

    #>> Cria uma legenda explicando as cores de acordo com o grau dos nós, é variável a cada vez que é rodado o código, pois depende da porcentagem

            #Definição da legenda
            legend_labels = [f"Grau ≤ {int(percentis[0])}", f"Grau {int(percentis[0])+1}-{int(percentis[1])}",
                             f"Grau {int(percentis[1])+1}-{int(percentis[2])}", f"Grau {int(percentis[2])+1}-{int(percentis[3])}",
                             f"Grau > {int(percentis[3])}"]
            
            cmap = plt.cm.plasma
            legend_colors = [cmap(0.1), cmap(0.3), cmap(0.5), cmap(0.7), cmap(0.9)]

            legend_elements = [Patch(facecolor=legend_colors[i], label=legend_labels[i], alpha=0.8) for i in range(5)]

            plt.legend(handles=legend_elements, loc="upper right", bbox_to_anchor=(1.15, 1.0), title="Legenda - Cores por Grau",
                  fontsize=10, framealpha=0.9)

            plt.title("REDE FACEBOOK", fontsize=16, pad=20)
            plt.axis("off")
            plt.tight_layout()
            plt.show()
            return True
            
        except Exception as e:
            print(f"Erro na visualização: {e}")
            return False
        
    #>> Método para poder visualizar as comunidades dentro do grafo
    def visualizar_comunidades(self):
        try:
            if self.G_subset is None:
                print("O subgrafo não existe!")
                return False

    #>> Mesma distribuição do método anterior

            pos = nx.spring_layout(self.G_subset, k=2, iterations=1000)

            #Calcula o grau de cada nó coloca em um dicionário e o tamanho do nó é proporcional ao grau
            graus = dict(self.G_subset.degree())
            tamanhos = [np.log(graus[node]+1)*25 for node in self.G_subset.nodes()]

    #>> As cores dos nós são a partir de uma lista dos valores das comunidades já calculadas anteriormente no método calcular_metricas()+1, pois as comunidades começam no 0, o que 
    #>> gera na visualização nós da mesma cor fazerem parte da mesma comunidade.

            cmap = plt.cm.get_cmap("plasma", max(self.communities.values())+1)
            
            #A cor de cada nó é a partir da lista de comunidades
            nx.draw_networkx_nodes(self.G_subset, pos, node_color=list(self.communities.values()), cmap=plt.cm.plasma, node_size=tamanhos, alpha=1)
            nx.draw_networkx_edges(self.G_subset, pos, alpha=1, edge_color="gray", width=0.5)

            plt.axis("off")
            plt.title("COMUNIDADES")
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Erro na visualização: {e}")
            return False
        
            
#Definição da main
def main():
    graph = FacebookGraph()

    if not graph.baixar_dados():
        return
    
    if not graph.carregar_rede():
        return
    
    if not graph.extrair_subconjunto(2000):
        return
    
    if not graph.calcular_metricas():
        return

    if not graph.visualizar_rede():
        return

if __name__ == "__main__":
    main()



#Etapa 5 - Relatório de análise
    #>> A análise relacionada à propriedade de cada nó ter maior influência, importância, potencial de espalhar informações ou outro 
    #>> está intrinsicamente relacionada às análises de centralidade. Dessa forma, como possuímos uma função para as centralidades, estaremos nos baseando nelas.
        #>> Grau de Centralidade:
            #>> Medida que indica o número de conexões diretas que um nó possui. Nós com grau elevado tendem a ser usuários populares, 
            #>> conectados diretamente a muitos outros — o que os torna importantes para disseminação imediata de informações. Os cinco nós com maior grau de centralidade obtiveram 
            #>> valores entre 0.04 e 0.03, em aproximação, o que indica que cada um se conecta a cerca de 70–80 nós diretamente, sendo assim, esses são nós de maior influência.
        #>> Centralidade de intermediação:
            #>> Medida que avalia quantas vezes um nó aparece nos caminhos mais curtos entre outros pares de nós. Representam usuários mediadores, que têm acesso mais rápido 
            #>> a informações de diferentes comunidades e controlam parte do fluxo comunicacional entre elas. Dessa forma, são nós com maior potencial de receber informação.
        #>> Centralidade de proximidade:
            #>> Medida da distância média de um nó até todos os outros, nós com alta proximidade conseguem alcançar rapidamente todos os demais da rede.
            #>> São nós centrais do ponto de vista estrutural, situados de forma estratégica com maior potencial de espalhar informação.
        #>> Centralidade de autovetor:
            #>> Um nó tem alta centralidade de autovetor se está conectado a outros nós influentes.
            #>> Informações iniciadas nesses nós têm maior probabilidade de atingir toda a rede, pois suas conexões diretas também possuem alta capacidade de difusão.
        #>> Mapeamento de Comunidades (Algoritmo de Louvain) — Estrutura Social da Rede
            #>> O algoritmo de Louvain identifica grupos densamente conectados dentro da rede. Cada comunidade representa um grupo social coeso, no qual as conexões internas 
            #>> são mais fortes que as externas — as “panelinhas” ou círculos de amizade da rede. Seu papel é compreender essas comunidades e permitir identificar subgrupos 
            #>> temáticos, interesses compartilhados ou bolhas informacionais.
            #>> Relação com as métricas: 
                    #>> Nós com alto grau tendem a ser centrais dentro de suas comunidades.
                    #>> Nós com alta intermediação frequentemente conectam diferentes comunidades.
                    #>> Nós com alta proximidade ou autovetor estão estrategicamente distribuídos entre comunidades.